# Simulations with TAMU data and [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

This is a basic simulation example using the [TAMU Cases](https://electricgrids.engr.tamu.edu/).

## Dependencies

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
using Dates

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations

### Optimization packages
For this simple example, we can use the Cbc solver with a relatively relaxed tolerance.

In [3]:
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Create a `System` from TAMU data
We have included some of the TAMU cases (with truncated time series info)
in the [PowerSystemsTestData](https://github.com/nrel-siip/powersystemstestdata)
repository for testing, so we can just use that.

In [4]:
PowerSystems.download(PowerSystems.TestData; branch = "master") # *note* add `force=true` to get a fresh copy
base_dir = dirname(dirname(pathof(PowerSystems)));

The TAMU data format relies on a folder containing `.m` or `.raw` files and `.csv`
files for the time series data. We have provided a parser for the TAMU data format with
the `TamuSystem()` fuction.

In [5]:
TAMU_DIR = joinpath(base_dir, "data", "ACTIVSg2000");
sys = TamuSystem(TAMU_DIR)
horizon = 24;
interval = Dates.Hour(24);
transform_single_time_series!(sys, horizon, interval);

┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/common.jl:25
┌ Info: Switched shunt converted to fixed shunt, with default value gs=0.0
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/psse.jl:312
┌ Info: angmin and angmax values are 0, widening these values on branch 2243 to +/- 60.0 deg.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/data.jl:1057
┌ Info: angmin and angmax values are 0, widening these values on branch 1881 to +/- 60.0 deg.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/data.jl:1057
┌ Info: angmin and angmax values are 0, widening these values on branch 1907 to +/- 60.0 deg.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/data.jl:1057
┌ Info: angmin and angmax values

## Run a PCM
note that the TAMU data doesn't contain startup and shutdown costs, or minimum up/down
time limits, so a UC problem merely respects minmum generation levels.

In [6]:
sim_folder = mkpath(joinpath(pkgpath, "TAMU-sim"))
stages_definition = Dict(
    "UC" => Stage(
        GenericOpProblem,
        template_unit_commitment(network = CopperPlatePowerModel),
        sys,
        solver,
    ),
)
order = Dict(1 => "UC")
horizons = Dict("UC" => 24)
intervals = Dict("UC" => (Hour(24), Consecutive()))
DA_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = IntraStageChronology(),
)

┌ Warning: No Feedforward Chronologies have been defined. This configuration assummes that there is no information passing between stages
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/core/simulation_sequence.jl:138


Feed Forward Chronology
-----------------------

Initial Condition Chronology
----------------------------

1



### Define and build a simulation

In [7]:
sim = Simulation(
    name = "TAMU-test",
    steps = 3,
    stages = stages_definition,
    stages_sequence = DA_sequence,
    simulation_folder = sim_folder,
)

build!(sim)

PowerSimulations.BuildStatusModule.BuildStatus.BUILT = 0

### Execute the simulation

In [8]:
execute!(sim)

Progress:  33%|█████████████▋                           |  ETA: 0:00:32
  Step:                  1
  Stage:                 UC
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 1.05921e+08 - 0.49 seconds
Cgl0004I processed model has 50536 rows, 51112 columns (38080 integer (38080 of which binary)) and 152112 elements
Cbc0045I Trying just fixing integer variables (and fixingish SOS).
Cbc0045I MIPStart solution provided values for 26088 of 38080 integer variables, 644 variables are still fractional.
Cbc0038I Full problem 50536 rows 51112 columns, reduced to 0 rows 0 columns
Cbc0045I Mini branch and bound defined values for remaining variables in 0.16 seconds.
Cbc0045I MIPStart provided solution with cost 1.05921e+08
Cbc0012I Integer solution of 1.0592117e+08 found by Reduced search after 0 iterations and 0 nod

PowerSimulations.RunStatusModule.RunStatus.SUCCESSFUL = 0

### Load and analyze results

In [9]:
sim_results = SimulationResults(sim);
uc_results = get_stage_results(sim_results, "UC")
read_realized_variables(uc_results, names = [:On__ThermalStandard])[:On__ThermalStandard]

┌ Info: checking integrity of /Users/cbarrows/Documents/repos/SIIPExamples.jl/TAMU-sim/TAMU-test/5/data_store/simulation_store.h5
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/utils.jl:386
┌ Info: Loaded time series from storage file existing=system-69c22ff2-721a-48bc-8cf9-af14c2052314_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_7rkHRR
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/i88mL/src/hdf5_time_series_storage.jl:82
┌ Info: reading variables from data store
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/core/simulation_results.jl:210


,generator-1004-1,generator-1006-1,generator-1009-1,generator-1011-1,generator-1021-1
,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0,1.0


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*